# Guide de pratique pour Label Studio

[Label Studio](https://labelstud.io/) est un outil d'annotation Open Source qui permet de façon flexible et collaborative de préparer un jeu de données et de valider des modèles AI. Il est possible d'accéder à Label Studio sur le serveur snoopy de la BnF, ou bien de l'installer localement sur sa machine.

## Confirguration et installation

````{tab} PIP
```{code-block}bash
pip install -U label-studio
label-studio
```
````
````{tab} BREW
```{code-block}bash
brew install heartexlabs/tap/label-studio
label-studio
```
````
````{tab} GIT
```{code-block}bash
git clone https://github.com/heartexlabs/label-studio.git
cd label-studio
pip install -e .
python label_studio/manage.py migrate
python label_studio/manage.py collectstatic
python label_studio/manage.py runserver
```
````
````{tab} DOCKER
```{code-block}bash
docker run -it -p 8080:8080 -v `pwd`/mydata:/label-studio/data heartexlabs/label-studio:latest
http://localhost:8080/
```
````

In [ ]:
#Installation avec un environnement Anaconda
conda create --name label-studio
conda activate label-studio
conda install psycopg2  # required for LS 1.7.2 only
pip install label-studio

In [ ]:
start label-studio

In [ ]:
#Installtio d'un SDK pour utiliser Label Studio avec Python
conda activate label-studio
pip install label-studio-sdk

```{admonition} snoopy
:class: snoopy
Pour se connecter sur Label Studio à partir d'un poste de la BnF, il suffit d'allez sur le serveur snoopy.
```

## Création d'un projet


Lors de votre première connexion il vous sera demandé de créer un compte nécessitant une adresse email et un mot de passe.
Une fois arrivé sur l'interface vous avez la possibilité de consulter un projet déjà crée, ou de créer le votre, pour ce faire il vous suffit de cliquer sur `create`.   
Chaque projet dispose d'un nom ainsi que d'une description (optionnelle), vous pouvez ensuite importer des fichiers de divers formats.   
Pour l'importation d'un grand nombre de fichiers il vous faudra passer par le **Cloud Storage**, nous y reviendrons par la suite.

Il vous seras également demandé de sélectionner un *template* pour l'interface, deux choix s'offre à vous: sélectionner un *template* déjà fournit par Label Studio où créer le votre, tout dépend de vos attentes en matière d'annotation. Le choix de *template* qu'offre Label Studio peut être limité, si vous souhaitez par exemple annoter du texte mais également des images il est sans doute préférable de créer votre propre interface.

```{image} ./assets/create-project.gif
:width: 1200px
:align: center
```

### via l'Inferface

Vous pouvez créer votre interface directement lors de la création de votre projet ou bien modifier et créer celui ci depuis  le bouton `settings` en haut à droite.

```{important}
:class: dropdown
Une fois votre *template* créer, il est déconseillé de modifier celui-ci car l'ensemble de vos données annoter reposerons sur ce *template* donc soyez prudent et prenez le temps de construire celui-ci en accord avec vos attentes en termes d'annotation.
```


```{image} ./assets/labelling-interface.gif
:width: 1200px
:align: center
```
Depuis l'onglet `Labeling Interface` vous pouvez ajouter un ensemble de label mais également coder votre interface depuis l'onglet `code`.  
Label Studio utilise un language XML.
Il existe trois différent types de familles de *tags* pour contrôler et intéragir avec votre interface: `Object` `Control` `Visual`. L'ensemble des tags sont disponible [ici](https://labelstud.io/tags/#Create-a-custom-labeling-configuration) Ci-dessous un exemple de code qui permet l'annotation de divers zone graphique et textuelle à travers trois types d'outils, l'ellipse, le polygone et le rectangle.   
Ce qu'il faut retenir c'est que la balise `<view>` agit comme une `<div>` en HTML et qu'elle est l'élément parent de l'ensemble des sous-balises.

```{code-block} xml
:linenos:
:name: codetemplatelabelstudio
:caption: Extrait d'un code XML pour l'interface Label Studio
<View>
<Image name="image" value="$image"/>  
<Header value="zone graphique"/>
<Labels name="type" toName="image">
<Label value="estampe" background="#ff0000"/>
<Label value="photographie" background="#007bff"/>
<Label value="dessin" background="#fb00ff"/>
<Label value="décoration" background="#8589ff"/>
<Label value="timbre" background="#ff6bd8"/>
<Label value="tampon" background="#FFA39E"/></Labels>
<Header value="zone textuelle"/>
<Labels name="transcription" toName="image">
<Label value="écriture manuscrite" background="#00ff4c"/>
<Label value="écriture typographique" background="#a8b404"/>
</Labels>
<Rectangle name="bbox" toName="image" strokeWidth="3"/>
<Polygon name="poly" toName="image" strokeWidth="3"/>
<Ellipse name="ellipse" toName="image"/>
</View>
```

### via le SDK Label Studio

Il vous faudra votre *TOKEN* afin d'utiliser le SDK. Celui-ci est disponible en cliquant sur la *tab* utilisateur en haut à droite.

```{image} ./assets/tabuser.png
:width: 400px
:align: center
```
```{image} ./assets/accesstoken.png
:width: 600px
:align: center
```

In [48]:
from label_studio_sdk.client import Client
from label_studio_sdk.project import Project
import requests
import os

In [35]:
URL = 'http://localhost:8080'
API_KEY = '36908b5e712c188d28c5b94bcda2f5209cf2ec93'
path_storage = 'C:\\Users\\gusta\\Desktop\\gallicapix_images\\images'

In [36]:
templates = """
<View>
<Image name="image" value="$image"/>  
<Header value="zone graphique"/>
<Labels name="type" toName="image">
<Label value="estampe" background="#ff0000"/>
<Label value="photographie" background="#007bff"/>
<Label value="dessin" background="#fb00ff"/>
<Label value="décoration" background="#8589ff"/>
<Label value="timbre" background="#ff6bd8"/>
<Label value="tampon" background="#FFA39E"/></Labels>
<Header value="zone textuelle"/>
<Labels name="transcription" toName="image">
<Label value="écriture manuscrite" background="#00ff4c"/>
<Label value="écriture typographique" background="#a8b404"/>
</Labels>
<Rectangle name="bbox" toName="image" strokeWidth="3"/>
<Polygon name="poly" toName="image" strokeWidth="3"/>
<Ellipse name="ellipse" toName="image"/>
</View>
"""

In [37]:
#Fonction qui vérifie la connexion et qui crée un obj Client
def check_connection(url, access_token):
    ls = Client(url,access_token)
    checking, = ls.check_connection().values()
    if checking == 'UP':
        return ls 
    else: 
        print('La connection à échouer. Vérifier le domain où la clée')

In [39]:
#Fonction qui crée un projet, si celui-ci existe déjà il retourne l'identifiant du projet
def get_or_set_project(client,**kwargs):
	kwargs['label_config'] = templates

	projects = client.list_projects()
	list_projects = [obj.get_params() for obj in projects]


	for obj in list_projects:
		if kwargs['title'] == obj['title']:
			url = client.get_url('projects') + "/" + str(obj['id'])
			print(f'{"[GET]":10}|{obj["title"]}\n{"":10}|par {obj["created_by"]["email"]:5}\n{"":10}|{url}\n')
			return client.get_project(obj['id'])
		else: 
			post_project = client.start_project(**kwargs)
			post_params = post_project.get_params()
			url = client.get_url('projects') + "/" + str(post_params['id'])
			print(f'{"[POST]":10}|{kwargs["title"]}\n{"":10}|par {post_params["created_by"]["email"]}\n{"":10}|{url}\n')
			return post_project

In [41]:
client = check_connection(URL, API_KEY)
project = get_or_set_project(client, title='demo_label-studio')

[POST]    |demo_label-studio
          |par mkdir.cultural.analytics@gmail.com
          |http://localhost:8080/projects/34



## Importation local de donnée via Cloud Storage

### via l'Interface

Pour l'importation d'un grand nombre de donnée il est nécessaire de passer par le *Cloud Storage* de Label Studio qui permet de synchroniser des données présente localement sur votre machine ou à distance par biais d'une base de données.   
Pour utiliser l'interface il vous suffit d'allez dans `Settings` et `Cloud Storage` et cliquer sur `Add Source Storage`.  
Il vous faudra également ajouter des variables d'environnement si cela n'est pas déjà fait. Pour indiquer le chemin du dossier que vous souhaitez synchroniser et activer le service de synchronisation.

In [ ]:
export LABEL_STUDIO_LOCAL_FILES_SERVING_ENABLED=true
export LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT=/home/user 
#Pour windows
setx LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT=C:\\data\\media

```{admonition} snoopy
:class: snoopy
Sur le serveur snoopy le dossier est synchronisé à l'adresse suivante: `LABEL_STUDIO_FILES_DOCUMENT_ROOT=/data/images_gallicapix`
```

```{image} ./assets/cloud-storage.png
:width: 600px
:align: center
```

Vous pouvez ensuite créer un *storage* en lui attribuant un nom et en précisant le chemin du dossier à synchroniser, celui-ci doit être un enfant de la variable `LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT` *(vous pouvez également filtrer les fichiers avec une regex)*. Cliquez ensuite sur `Add Storage` et `Sync Storage` pour importer ou synchroniser les fichiers.

### via le SDK Label Studio

In [1]:
# Fonction qui crée un storage ci celui-ci n'existe pas. Il récupère son idendifiant dans le cas contraire.
def get_or_set_local_storage(project, **kwargs):

	headers = headers = {'Authorization': f'Token {API_KEY}'}
	id_ = str(project.get_params()["id"])
	params = (('project', id_),)	
	response = requests.get(f'{URL}/api/storages/localfiles/', headers=headers, params=params)

	list_storages = []
	if response.status_code == 200:
		list_storages = response.json()
	
	for obj in list_storages:
		if obj['path'] == kwargs['local_store_path']:
			print(f"{'[GET]':10}|<{obj['path']}>\n{'':10}|crée le {parser.parse(obj['created_at']).strftime('%d/%m/%Y %H:%M')}\n")
			return obj
	else:
 		storage = project.connect_local_import_storage(**kwargs)
 		print(f"{'[POST]':10}|ajoute le localstorage <{kwargs['local_store_path']}>")
 		return storage

In [45]:
path_storage = 'C:\\Users\\gusta\\Desktop\\gallicapix_images\\images'
storage = get_or_set_local_storage(project,local_store_path=path_storage)

[POST]    |ajoute le localstorage <C:\Users\gusta\Desktop\gallicapix_images\images>


In [2]:
#Fonction pour synchroniser le 'storage'
def sync_local_storage(client, storage):
	id_ = storage['id']
	type_ = storage['type']
	files = len([image for root, dirs, file  in os.walk(storage['path']) for image in file])
	last_sync = f'scan précédent le {parser.parse(storage["last_sync"]).strftime("%d/%m/%Y %H:%M")}' if storage['last_sync'] else ''

	print(f"{'[SYNC]':10}|{storage['path']}\n{'files':10}|{files} fichiers présent dans le {type_}\n{'scan..':10}|{last_sync}")
	sync =  client.sync_storage(type_, id_)
	print(f"{'add':10}|{sync['last_sync_count']} fichiers\n")

In [49]:
#Peut-être long dépend du nombre de données à synchroniser
sync_local_storage(client,storage)

[SYNC]    |C:\Users\gusta\Desktop\gallicapix_images\images
files     |13416 fichiers présent dans le localfiles
scan..    |
add       |13416 fichiers



## Annotation

Pour l'interface d'annotation il est possible de créer différent onglets avec différent filtres afin d'organiser l'ensemble de ces données.   
Lors du téléchargement des images avec le module [`iiif_from_csv.collecting_image()`](namefordoc) il est possible d'attribuer à chaque image un `name` qui permet ainsi de créer des classes d'images en plus des *form* IIIF présentes dans le nom de fichier.  
Tout cela réunit permet de contrôler de manière très fine le type d'images que l'on souhaite filtrer *(il existe également d'autres types de filtres au délà du nom de l'image).*

```{image} ./assets/create-tab.gif
:width: 1100px
:align: center
```
Pour commencer l'annotation il suffit de cliquer sur une image et sélectionner un outil d'annotation ainsi qu'un label. Plusieurs filtres sont également disponible en fonction de la méthodologie adoptée. Une fois l'annotation achevée, cliquer sur le bouton `submit` afin de valider celle-ci. Une date d'annotation seras alors attribué à l'image ainsi que son annotateur. 

```{image} ./assets/do_annotation.gif
:width: 1100px
:align: center
```


### Exportation des annotations

Il existe plusieurs format d'exportation, le format `.json` est le format par défaut de Label Studio, il incorpore l'ensemble des métadonnées issues de l'application ainsi que les informations de segmentation.
Les quatres clées essentielles à retenir ici sont: 
1. `id` qui indique l'identifiant unique et recherchable pour l'image en question à l'intérieur de l'application. 
2. `annotations` qui regroupe l'ensemble des métadonnées concernant les annotations, la clée `result` contient une liste d'annotations
3. `predictions` qui regroupe l'ensemble des prédications *(quand un modèle est lancé depuis l'application)* 
4. `data` qui indique le chemin de la donnée *(ici une image)*

```{code-block} json
:linenos:
:emphasize-lines: 2,3,7,26,27,13
:caption: Exemple de métadonnée associées à une image
{
"id": 158833,
"annotations":[
    {
    "id": 2801,
    "completed_by": 1,
    "result":[],
    "was_cancelled": false,
    "ground_truth": false,
    "created_at": "2023-05-06T22:34:35.346372Z",
    "updated_at": "2023-05-06T22:34:35.346372Z",
    "lead_time": null,
    "prediction":{},
    "result_count": 0,
    "unique_id": "0f4a00d7-cc55-46f4-8c51-75e57ffc60f7",
    "last_action": null,
    "task": 158833,
    "project": 30,
    "updated_by": 1,
    "parent_prediction": null,
    "parent_annotation": null,
    "last_created_by": null
    }
],
"drafts":[],
"predictions":[],
"data":{"image": "/data/local-files/?d=Users%5Cgusta%5CDesktop%5Cgallicapix_images%5Cimages%5Caffiche%5Cbpt6k6964173h%24f1%24full%24%211212_777%240%24native_affiche.jpg"},
"meta":{},
"created_at": "2023-05-06T22:18:16.976585Z",
"updated_at": "2023-05-06T22:34:35.385910Z",
"inner_id": 1,
"total_annotations": 1,
"cancelled_annotations": 0,
"total_predictions": 0,
"comment_count": 0,
"unresolved_comment_count": 0,
"last_comment_updated_at": null,
"project": 30,
"updated_by": 1,
"comment_authors":[]
}
```

In [3]:
import json
export_labelstudio = open('./data/labelstudio/project-30-at-2023-05-09-16-12-caaecfb1.json', 'r', encoding='utf8')
dataset = json.load(export_labelstudio)
print(len(dataset)) #Nombre d'images présent dans jeu de donnée

750


In [4]:
dataset[0]['id'],dataset[0]['data'],dataset[0]['predictions']

(158833,
 {'image': '/data/local-files/?d=Users%5Cgusta%5CDesktop%5Cgallicapix_images%5Cimages%5Caffiche%5Cbpt6k6964173h%24f1%24full%24%211212_777%240%24native_affiche.jpg'},
 [])

In [7]:
#Métadonnée concernant la première annotation de la première image
dataset[0]['annotations'][0]['result'][0]

{'original_width': 348,
 'original_height': 544,
 'image_rotation': 0,
 'value': {'x': 16.761363636363637,
  'y': 9.25589836660617,
  'width': 69.60227272727273,
  'height': 89.11070780399274,
  'rotation': 0},
 'id': 'FB7KgtTRmh',
 'from_name': 'bbox',
 'to_name': 'image',
 'type': 'rectangle',
 'origin': 'manual'}

### Importation des annotations (depuis une autre application)

Dans le cas éventuel ou plusieurs personnes travailleraient ensemble sur des appplications différentes, il peut-être difficile d'importer et de synchroniser l'ensemble des données. 
Le serveur snoopy de la BnF reste l'application de référence en terme de collaboration, dans le cas contraire ou une personne aimerais importer des annotations sur des données qui se trouvent déjà présente sur l'application Label Studio de snoopy. Il est possible via le SDK d'importer des images.

Ici un exemple d'une solution de contournement, qui repose et qui fonctionne uniquement parce que l'image est de même dimension et que l'ark et l'instance de l'image sont présent dans les deux noms de fichiers. Si des images sont distribuer sur plusieurs applications il est conseillé d'utiliserle module [`iiif_from_csv.collecting_image()`](namefordoc) car le nom des fichiers sont formattés de façon précise et fine.

In [9]:
import re
from urllib.parse import unquote
#Permet de parse un URL si le fichier comprend des caractères qui ont été normaliser par Label Studio
url = unquote(dataset[0]['data']['image'])
print(dataset[0]['data']['image'])
print(url)

/data/local-files/?d=Users%5Cgusta%5CDesktop%5Cgallicapix_images%5Cimages%5Caffiche%5Cbpt6k6964173h%24f1%24full%24%211212_777%240%24native_affiche.jpg
/data/local-files/?d=Users\gusta\Desktop\gallicapix_images\images\affiche\bpt6k6964173h$f1$full$!1212_777$0$native_affiche.jpg


In [10]:
#Nettoyage de la chaîne de caractère
url_norm = url.split('\\')[-1].replace('$','')
url_norm

'bpt6k6964173hf1full!1212_7770native_affiche.jpg'

In [11]:
#Regex qui récupère l'ark et l'instance pour l'image en question 'f1', cette information peut nous servir de clée de jointure
re.findall(r'(.*)full',url_norm)[0]

'bpt6k6964173hf1'

In [114]:
'''
Un exemple de fonction qui peut être utilisée pour retrouver les images présent sur l'application,
en utilisant l'ark et l'instance de l'image. 
Note: La recherche se fait de façon séquentielle et n'est sans pas la plus optimale
'''
def add_annotation_from_extern(project,external_source):
    #REGEX
    def regexcompiler(str, regex):
        return re.findall(r'(.*)full', str)[0]
    list_tasks = project.tasks # requête l'application afin de récupérer l'ensemble des tasks (image/métadonnée)
    # Créer une liste avec tuple (ark/image, result)
    ext_id_annotation = [(regexcompiler(unquote(obj['data']['image']).split('\\')[-1].replace('$',''),regex_out), obj['annotations'][0]["result"]) for obj in external_source]
    adding = 0
    #Boucle autour des tacks présent dans l'application
    for obj in list_tasks:
        #Décode le chemin URL de chaque tasks et le reformate
        decode = unquote(obj['data']['image']).split('\\')[-1].replace('$','')
        #Récupère l'ark/image
        ark = regexcompiler(decode,r'(.*)full')
        #Boucle autours des tasks présent dans le dataset
        for id_, result_ in ext_id_annotation:
            #Vérifie que les tasks ne comporte pas d'annotations et que la jointure entre les deux arks/image est 'true'
            if (len(obj['annotations']) == 0) and (ark == id_):
                #Incrémente le compteur
                adding += 1
                #Ajoute les annotations
                print(f"{'[POST]':10}|annotation pour l'id: {obj['id']} ark: {ark}")
                project.create_annotation(task_id=obj['id'], result=result_)
    print(f"{'resume':10}|{adding} annotations ajoutées")

In [115]:
add_annotation_from_extern(project, dataset)

[POST]    |annotation pour l'id: 172249 ark: bpt6k6964173hf1
[POST]    |annotation pour l'id: 172250 ark: bpt6k69646507f1
[POST]    |annotation pour l'id: 172251 ark: bpt6k69646893f1
[POST]    |annotation pour l'id: 172252 ark: bpt6k9764501jf1
[POST]    |annotation pour l'id: 172333 ark: btv1b100501693f1
[POST]    |annotation pour l'id: 172337 ark: btv1b10050505bf1
[POST]    |annotation pour l'id: 172339 ark: btv1b10051224xf1
[POST]    |annotation pour l'id: 172340 ark: btv1b10051250cf1
[POST]    |annotation pour l'id: 172341 ark: btv1b100512797f1
[POST]    |annotation pour l'id: 172345 ark: btv1b10052973jf1
[POST]    |annotation pour l'id: 172346 ark: btv1b100529812f1
[POST]    |annotation pour l'id: 172347 ark: btv1b10052985vf1
[POST]    |annotation pour l'id: 172348 ark: btv1b100530130f1
[POST]    |annotation pour l'id: 172350 ark: btv1b10053438kf1
[POST]    |annotation pour l'id: 172351 ark: btv1b100534480f1
[POST]    |annotation pour l'id: 172388 ark: btv1b10101472kf1
[POST]    |a